# Отбор признаков
Продемонстрируем работу методов отбора признаков на примере предсказания отклика на рекламную кампанию. Считаем данные и посмотрим на их содержание.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_excel("train_sample.xlsx")

In [ ]:
data.head()

In [ ]:
data.columns

Удалим пропуски в данных, выделим целевую переменную и разобьём данные на train и test

In [ ]:
data.dropna(inplace=True)

In [ ]:
y = data['Целевая. Логарифм дохода клиента'].values

In [ ]:
X = data.drop(columns=['Целевая. Логарифм дохода клиента', 'Целевая. Доход клиента'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

Матрица корреляций. Видим, что есть множество корреляций, поскольку изначально данные дублируются. Выделим колонки с логарифмами признаков.

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(X_train.corr())

In [ ]:
basic_columns = X.columns[:-8].values
log_columns = ['ID клиента',
                         'Сумма всех приходных операций',
                         'Атрибут. Возраст', 
                         'Атрибут. Пол',
                         'Атрибут. Тип населенного пункта: города-миллионеры',
                         'Атрибут. Тип населенного пункта: остальные города',
                         'Атрибут. Гражданин РФ', 'Атрибут. Сотрудник Сбербанка',
                         'Атрибут. Сегмент ВИП',
                         'Атрибут. Сегмент МВС (массовый высокодоходный)',
                         'Атрибут. Социальный сегмент', 'Атрибут. Массовый сегмент',
                         'Атрибут. Активный мобильный банк', 'Атрибут. Наличие Пакета Услуга',
                         'Атрибут. Уровень утилизации кредитной карты',
                         'Атрибут. Подключенная программа лояльности Спасибо',
                         'Логарифм. Сумма входящих переводов по картам',
                         'Логарифм. Сумма снятий наличных в банкоматах',
                         'Логарифм. Сумма трат в категории Кафе/Рестораны',
                         'Логарифм. Средний ежемесячный платеж по кредитам',
                         'Логарифм. Основной долг по ипотеке',
                         'Логарифм. Основной долг по потребительским кредитам',
                         'Логарифм. Сумма соц начислений',
                         'Логарифм. Суммарный баланс по всем счетам', 
                         'Логарифм. P&L клиента']


Обучим два бейзлайна: Ridge-регрессию и случайный лес. Измерим качество и визуализируем коэффициенты признаков.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

In [ ]:
cross_val_score(Ridge(), X=X_train[log_columns], y=y_train, cv=5, scoring='r2').mean(), \
cross_val_score(Ridge(), X=X_train, y=y_train, cv=5, scoring='r2').mean()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
cross_val_score(RandomForestRegressor(n_estimators=100), X=X_train[log_columns], y=y_train, cv=5, scoring='r2').mean(), \
cross_val_score(RandomForestRegressor(n_estimators=100), X=X_train, y=y_train, cv=5, scoring='r2').mean()

In [ ]:
model = Ridge().fit(X_train[log_columns], y_train)
plt.figure(figsize=(20, 8))
plt.bar(log_columns, model.coef_)

In [ ]:
list(zip(model.coef_, log_columns))

## Модели на основе статистического анализа
sklearn содержит в себе множество статистических тестов для определения важности признаков.

Для задачи регрессии: f_regression, mutual_info_regression
Для задачи классификации: chi2, f_classif, mutual_info_classif

Воспользуемся методом mutual_info_regression.

In [ ]:
from sklearn.feature_selection import mutual_info_regression

mutual_information = mutual_info_regression(X_train[log_columns], y_train)

In [ ]:
mutual_information

In [ ]:
plt.figure(figsize=(20, 8))
plt.bar(log_columns, mutual_information)

In [ ]:
from sklearn.feature_selection import GenericUnivariateSelect

In [ ]:
trans = GenericUnivariateSelect(score_func=mutual_info_regression, mode='percentile', param=50)
X_trans = trans.fit_transform(X_train[log_columns], y_train)

In [ ]:
columns_retained_Select = np.array(log_columns)[trans.get_support()]
X_trans = pd.DataFrame(X_trans, columns=columns_retained_Select)
X_trans.head()

In [ ]:
cross_val_score(Ridge(), X=X_trans, y=y_train, cv=5, scoring='r2').mean()


Sklearn также поддерживает возможность выбрать признаки исходя из их важности в моделях.

In [ ]:
from sklearn.feature_selection import SelectFromModel

clf = RandomForestRegressor()
trans = SelectFromModel(clf, threshold='median')
X_trans = trans.fit_transform(X_train[log_columns], y_train)

columns_retained_Select = np.array(log_columns)[trans.get_support()]
X_trans = pd.DataFrame(X_trans, columns=columns_retained_Select)
X_trans.head()

In [ ]:
cross_val_score(Ridge(), X=X_trans, y=y_train, cv=5, scoring='r2').mean()

Рекурсивный поиск важных признаков позволяет выбрать признаки, которые дают наилучшее качество. В стандартном (не рекурсивном) варианте можно фиксировать количество желаемых признаков.

In [ ]:
from sklearn.feature_selection import RFECV

clf = Ridge()
trans = RFECV(clf)
X_trans = trans.fit_transform(X_train[log_columns], y_train)
columns_retained_Select = np.array(log_columns)[trans.get_support()]

X_trans = pd.DataFrame(X_trans, columns=columns_retained_Select)
X_trans.head()

In [ ]:
cross_val_score(Ridge(), X=X_trans, y=y_train, cv=5, scoring='r2').mean()

In [ ]:
columns_retained_Select.shape, len(log_columns)

In [ ]:
from sklearn.feature_selection import RFE

clf = Ridge()
trans = RFE(clf, n_features_to_select=13)
X_trans = trans.fit_transform(X_train[log_columns], y_train)
columns_retained_Select = np.array(log_columns)[trans.get_support()]

X_trans = pd.DataFrame(X_trans, columns=columns_retained_Select)
X_trans.head()

## Выводы